# TP 2 Matias Gathoye avec l'année 1898

## Partie Keyword

### Préparation de l'extraction

In [ ]:
import os
import yake

# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

### Choix de l'année et sélection des fichiers

In [ ]:
# Choisir une année
year = 1898

# Lister les Fichiers
txt_path = '../data/txt'
files = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]

### Extraction des keywords de tous les fichiers 
(attention, pour 100 fichiers c'est long. Je ne l'ai pas fait mais j'ai vu que ça fonctionnait)

In [ ]:
for f in sorted(files):
    text = open(os.path.join(txt_path, f), 'r', encoding="utf-8").read()
    keywords = kw_extractor.extract_keywords(text)
    kept = []
    for kw, score in keywords:
        words = kw.split()
        if len(words) == 2:
            kept.append(kw)
    print(f"{f} mentions these keywords: {', '.join(kept)}...")

## Partie Wordcloud
Il y a des lignes qui seront répétées par rapport au code-ci dessus, comme la déclaration de l'année par exemple. Je les laisse au cas où vous commencez à lancer le code à partir du point 2. Ce sera pareil pour les points suivants.

### Imports et stopwords

In [ ]:
from collections import Counter
from wordcloud import WordCloud
import os  
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from IPython.display import Image

In [ ]:
# Stopwords (Idem que dans s1)
sw = stopwords.words("french")
sw += ["les", "plus", "cette", "fait", "faire", "être", "deux", "comme", "dont", "tout",
       "ils", "bien", "sans", "peut", "tous", "après", "ainsi", "donc", "cet", "sous",
       "celle", "entre", "encore", "toutes", "pendant", "moins", "dire", "cela", "non",
       "faut", "trois", "aussi", "dit", "avoir", "doit", "contre", "depuis", "autres",
       "van", "het", "autre", "jusqu", "ville", "rossel", "dem"]

# liste perso en plus
sw += ["où", "apparemment", "décidément", "déjà", "alors", "toutes", "leurs", "avant", "celui", "toute", "elles",
       "dés", "très", "peu"]
sw = set(sw)

# permet de savoir si un mot est dans ma liste de sw
print("avant" in sw)

### Création du fichier avec le texte de tous les journaux de 1898

In [ ]:
# Choisir une année
year = 1898

# Lister les fichiers de cette année
data_path = '../data'
txt_path = '../data/txt'
txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]
len(txts)

# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r', encoding='utf-8') as f:
        content_list.append(f.read())

# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w', encoding='utf-8') as f:
    f.write(' '.join(content_list))

### Nettoyage du fichier

In [ ]:
def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, 'r', encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.upper() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [ ]:
clean_text(year, folder=temp_path)

# Vérifier le résultat
with open(os.path.join(temp_path, f'{year}_clean.txt'), 'r', encoding='utf-8') as f:
    after = f.read()

after[:500]

### Nuage de mots

In [ ]:
frequencies = Counter(after.split())

cloud = WordCloud(width=2000, height=1000, background_color='white').generate_from_frequencies(frequencies)
cloud.to_file(os.path.join(temp_path, f"{year}.png"))
Image(filename=os.path.join(temp_path, f"{year}.png"))

## Partie entité nommées

### Imports

In [14]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

nlp = spacy.load('fr_core_news_md')

### Application de la reconnaissance d'entités nommées pour l'année 1898

In [15]:
# Charger le texte
n=100000
text = open("../data/tmp/1898.txt", encoding='utf-8').read()[:n]

# Traiter le texte
doc = nlp(text)

### Changement du if pour récuperer les personnes mais aussi les organisations et les lieux

In [16]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if (ent.label_ == "PER" or ent.label_ == "ORG" or ent.label_ == "LOC") and len(ent.text) > 3:
        people[ent.text] += 1

In [17]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

Rossel apparait 20 fois dans le corpus
nn dem apparait 16 fois dans le corpus
Bruxelles apparait 12 fois dans le corpus
Nord apparait 9 fois dans le corpus
Agence Rossel apparait 8 fois dans le corpus
O n dem apparait 8 fois dans le corpus
Brux apparait 7 fois dans le corpus
Tournai apparait 5 fois dans le corpus
F ille apparait 5 fois dans le corpus
Louvain apparait 5 fois dans le corpus
Midi apparait 5 fois dans le corpus
Centre apparait 5 fois dans le corpus
Anvers apparait 4 fois dans le corpus
cerl apparait 4 fois dans le corpus
la France apparait 4 fois dans le corpus
Belgique apparait 4 fois dans le corpus
Ixellcs apparait 3 fois dans le corpus
Gand apparait 3 fois dans le corpus
Laeken apparait 3 fois dans le corpus
rue Gallait apparait 3 fois dans le corpus
prop apparait 3 fois dans le corpus
Londres apparait 3 fois dans le corpus
Laines apparait 3 fois dans le corpus
Discr apparait 3 fois dans le corpus
avenue Louise apparait 3 fois dans le corpus
Bourse apparait 3 fois dans 

## Analyse de sentiments avec Textblob-FR

### Imports

In [18]:
import sys
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer

### Reprise de la fonction `get_sentiment`

In [19]:
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())

def get_sentiment(input_text):
    blob = tb(input_text)
    polarity, subjectivity = blob.sentiment
    polarity_perc = f"{100*abs(polarity):.0f}"
    subjectivity_perc = f"{100*abs(subjectivity):.0f}"
    if polarity > 0:
        polarity_str = f"{polarity_perc}% positive"
    elif polarity < 0:
        polarity_str = f"{polarity_perc}% negative"
    else:
        polarity_str = "neutral"
    if subjectivity > 0:
        subjectivity_str = f"{100*subjectivity:.0f}% subjective"
    else:
        subjectivity_str = "perfectly objective"
    print(f"This text is {polarity_str} and {subjectivity_str}.")

### Analyse des 10 phrases de 1898 choisies arbitrairement

In [20]:
get_sentiment("On croit que ce boulevard recevra le nom d’Esterhazy.") #1

get_sentiment("Evidemment e’était mon droit, puisqu’il y avait menace, exhibition de J’arme.") #2
get_sentiment("Évidemment c’était mon droit, puisqu’il y avait menace, exhibition de l’arme.") 
# j'ai mis  la même phrase avec correction des petites fautes pour voir si ça changeait le résultat. Ce n'est pas le cas apparemment :)

get_sentiment("satisfait, il allait prononcer;: Ne bougeons plus! quand patatras, l’estrade mal assise s'effondre et les douze hommes s’étalent pèle môle parmi les planches et les trétaux. ") #3
get_sentiment("L'aventure eut été plutôt joyeuse si quatre malheureux n’étaient restés étendus, les jambes plus ou moins mises à mal.") #4
get_sentiment("Informations prises, les victimes avaient eu plus de peur que de mal, heureusement.") #5
get_sentiment("Ce n’est pas la première fois,souvenez-vous-en, que cette juridiction modeste et expéditive se signale à l’admiration sympathique du monde de la pédale.") #6
get_sentiment("Or, voici quels événements minuscules ont suscité la décision monumentale qui me chiffonne itérativement.") #7
get_sentiment("Non, mais c’est effrayant et désespérant tout ensemble de constater comme les plus précieux proverbes sont chaque jour foulés aux pieds ,par ceux qui les premiers devraient s’en souvenir") #8
get_sentiment("il n’y a plus de securité morale pour aucun étranger en France.") #9
get_sentiment("Je trouve que, sans exagération; ia Tisane des Shakers est un remède merveilleux.") #10

This text is neutral and perfectly objective.
This text is 18% positive and 20% subjective.
This text is 18% positive and 20% subjective.
This text is 5% negative and 35% subjective.
This text is 2% negative and 50% subjective.
This text is 8% negative and 47% subjective.
This text is 30% positive and 37% subjective.
This text is 4% positive and 35% subjective.
This text is 20% positive and 32% subjective.
This text is 8% positive and 13% subjective.
This text is 90% positive and 90% subjective.
